In [1]:
import sys
import numpy as np
import pandas as pd
import requests

def get_input(user_id):
    adress = 'http://35.180.178.243'

    req = requests.get(adress + '/reset?user_id=' + user_id)

    data = req.json()

    nb_users = data.pop('nb_users')
    nb_items = data.pop('nb_items')
    next_state = data.pop('next_state')
    columns = ['state_history', 'action_history', 'rewards_history']
    data_array = np.array([data.pop(c) for c in columns])
    df =  pd.DataFrame(data_array.T, columns=columns)
    return nb_users, nb_items, next_state, data_array

In [2]:
ID = 'VI2X71V0287S9F9B7SCU'

In [3]:
k = 10
t = []
for i in range(k):
    _, _, _ , data = get_input(ID)
    _, _, reward_history = data
    nb_reco = len(reward_history)
    good_reco = reward_history > 0
    nb_good_reco = np.sum(good_reco)
    t.append(100*nb_good_reco/nb_reco)

print('Distribution du taux de bonnes recommandations sur {} resets :'.format(k), t)
print('Taux moyen :', np.mean(t))
print('Ecart type :', np.std(t))

Distribution du taux de bonnes recommandations sur 10 resets : [29.0, 32.0, 29.5, 29.0, 30.5, 28.0, 35.0, 27.5, 32.0, 33.5]
Taux moyen : 30.6
Ecart type : 2.33238075793812


In [4]:
nb_users, nb_items, next_state , data = get_input(ID)

state_history, action_history, reward_history = data
columns_names = ['user_id', 'item_id', 'item_price'] + ['metadata_' + str(i) for i in range(5)]
dfs = [pd.DataFrame(state_history[i], columns=columns_names) for i in range(len(state_history))]
df = pd.DataFrame(state_history[0], columns=columns_names)
df['hist_line'] = 0
df['action_item_id'] = state_history[0][action_history[0]][1] 
df['reward'] = reward_history[0]
for i in range(1, len(state_history)):
    df_temp = pd.DataFrame(state_history[i], columns=columns_names)
    df_temp['hist_line'] = i
    df_temp['action_item_id'] = state_history[i][action_history[i]][1] 
    df_temp['reward'] = reward_history[i]
    df = df.append(df_temp, ignore_index=True)

df_next_state = pd.DataFrame(next_state, columns=columns_names)
user_df = pd.DataFrame(np.arange(nb_users), columns=['user_id']).set_index('user_id')
item_df = pd.DataFrame(np.arange(nb_items), columns=['item_id']).set_index('item_id')
df.head()

print('Nombre d\'utilisateurs :', nb_users)
print('Nombre d\'items :', nb_items)

nb_reco = len(reward_history)
good_reco = reward_history > 0
nb_good_reco = np.sum(good_reco)
nb_bad_reco = nb_reco - nb_good_reco
print('Nombre de recommandations :', nb_reco)
print('Nombre de bonnes recommandations :', nb_good_reco)
print('Nombre de mauvaises recommandations :', nb_bad_reco)
print('Taux de bonnes recommandations : {}%'.format(100*nb_bad_reco/nb_reco))

Nombre d'utilisateurs : 100
Nombre d'items : 30
Nombre de recommandations : 200
Nombre de bonnes recommandations : 64
Nombre de mauvaises recommandations : 136
Taux de bonnes recommandations : 68.0%


## Utilisateurs/items non présents dans l'historique

In [5]:
print('Nombre d\'utilisateurs non présents', nb_users-df['user_id'].nunique())
print('Nombre d\'items non présents', nb_items-df['item_id'].nunique())

Nombre d'utilisateurs non présents 14
Nombre d'items non présents 0


## Metadonnées utilisateur

In [6]:
for i in range(5):
    print('Metadata', i, ': ', (df.groupby('user_id')['metadata_' + str(i)].nunique() > 1).sum() == 0)

Metadata 0 :  True
Metadata 1 :  True
Metadata 2 :  False
Metadata 3 :  False
Metadata 4 :  False


Les metadonnées 0 et 1 sont des métadonnées utilisateurs (d'après l'historique)

## Metadonnées produits

In [7]:
for i in range(5):
    print('Metadata', i, ': ', (df.groupby('item_id')['metadata_' + str(i)].nunique() > 1).sum() == 0)

Metadata 0 :  False
Metadata 1 :  False
Metadata 2 :  True
Metadata 3 :  True
Metadata 4 :  False


Les metadonnées 2 et 3 sont des métadonnées utilisateurs (d'après l'historique)

## Nombre de commandes par utilisateurs

In [8]:
print('Nombre de commande par utilisateur minimum :', df.groupby('user_id')[['hist_line']].count().min()[0]) # 0 si on compte les absents
print('Nombre de commande par utilisateur maximum :', df.groupby('user_id')[['hist_line']].count().max()[0])

Nombre de commande par utilisateur minimum : 30
Nombre de commande par utilisateur maximum : 232


## Diversité des achats (quantitative)

In [9]:
df_bought = df[df.reward > 0]
df_bought = df_bought[df_bought.action_item_id == df_bought.item_id]
diversity = df_bought.groupby('user_id')[['item_id']].nunique()
number = df_bought.groupby('user_id')[['item_id']].nunique()

In [10]:
print('Nombre d\'articles par utilisateur minimum :', df.groupby('user_id')[['item_id']].nunique().min()[0]) # 0 si on compte les absents

Nombre d'articles par utilisateur minimum : 30


## Rapport au prix

In [8]:
#df[['user_id', 'item_price', 'metadata_2', 'metadata_3', 'metadata_4']].groupby('user_id').describe()
print('Minimum de la dépense moyenne :', df[['user_id', 'item_price']].groupby('user_id').mean().min()[0])
print('Maximum de la dépense moyenne :', df[['user_id', 'item_price']].groupby('user_id').mean().max()[0])
print('Minimum de l\'ecart type :', df[['user_id', 'item_price']].groupby('user_id').std().min()[0])
print('Maximum de l\'ecart type :', df[['user_id', 'item_price']].groupby('user_id').std().max()[0])


Minimum de la dépense moyenne : 478.6666994958898
Maximum de la dépense moyenne : 497.6739666563779
Minimum de l'ecart type : 285.27444027112324
Maximum de l'ecart type : 295.3532546375919


## Apparition de nouveaux utilisateurs ?

In [12]:
def send_pred(user_id, pred):
    adress = 'http://35.180.178.243'
    req = requests.get(adress + '/predict?user_id=' + user_id + '&recommended_item=' + str(pred))

    data = req.json()
    reward = data.pop('reward')
    state = data.pop('state')
    return reward, state

In [13]:
user_df = df.drop_duplicates(subset=['user_id']).sort_values('user_id')[['user_id', 'metadata_0', 'metadata_1']]

In [14]:
from numpy import linalg

norm = linalg.norm(user_df.values, axis = 1)
user_df['metadata_0_normed'] = user_df.metadata_0.values / norm
user_df['metadata_1_normed'] = user_df.metadata_1.values / norm

In [15]:
user_metadata = user_df[['metadata_0_normed', 'metadata_1_normed']].values

In [37]:
user_idx = 15
user_id = user_df.user_id.iloc[user_idx]
sim = user_metadata.dot(user_metadata[user_idx])
sim[user_idx] = -1
similar_to_user = np.argmax(sim)
user_df.user_id.iloc[similar_to_user]

5

In [34]:
print('Utilisateur {} similaire a {}'.format(similar_to_user, user_id))

Utilisateur 4 similaire a 21


In [48]:
def get_sim(user_df, user_idx):
    norm = linalg.norm(user_df.values, axis = 1)
    user_df['metadata_0_normed'] = user_df.metadata_0.values / norm
    user_df['metadata_1_normed'] = user_df.metadata_1.values / norm
    user_metadata = user_df[['metadata_0_normed', 'metadata_1_normed']].values
    user_id = user_df.user_id.iloc[user_idx]
    sim = user_metadata.dot(user_metadata[user_idx])
    sim[user_idx] = -1
    similar_to_user = np.argmax(sim)
    similar_user_id = user_df.user_id.iloc[similar_to_user]
    return user_id, similar_user_id 

In [49]:
get_sim(df_bought, 10)

(24, 48)

In [50]:
df_bought[df_bought.user_id == 24]

,user_id,item_id,item_price,metadata_0,metadata_1,metadata_2,metadata_3,metadata_4,hist_line,action_item_id,reward,metadata_0_normed,metadata_1_normed
1633,24,21,246.886226,0.958731,2.312526,1.925705,1.393613,0.536742,54,21,246.886226,0.002698,0.006507


In [51]:
df_bought[df_bought.user_id == 48]

,user_id,item_id,item_price,metadata_0,metadata_1,metadata_2,metadata_3,metadata_4,hist_line,action_item_id,reward,metadata_0_normed,metadata_1_normed
2013,48,12,88.726188,2.650728,1.769263,1.058451,1.234963,0.957324,67,12,88.726188,0.01754,0.011707


In [70]:
df_bought[['user_id', 'item_df_id']].groupby(['user_id']).nunique()[['item_id']] == df_bought[['user_id', 'item_id']].groupby(['user_id']).count()

,item_id
user_id,
1,True
5,True
9,True
11,True
15,True
17,True
21,True
22,True
24,True
